<a href="https://colab.research.google.com/github/ShlokM08/CSE508_Winter2024_A2_2021421./blob/main/IRASSIGNMENT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#IMPORT DRIVE
from google.colab import drive
drive.mount('/content/drive')
!pip install python-docx nltk


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.5 MB/s eta 0:00:00


In [7]:
import pandas as pd
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import pickle
import numpy as np
import os

class ImageProcessor:
    def __init__(self):
        self.transform_pipeline = transforms.Compose([
            transforms.Resize(299),
            transforms.CenterCrop(299),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.model = models.inception_v3(pretrained=True)
        self.model.fc = torch.nn.Identity()
        self.model.eval()

    def fetch_and_process(self, url):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                with Image.open(BytesIO(response.content)).convert("RGB") as img:
                    img_tensor = self.transform_pipeline(img).unsqueeze(0)
                    with torch.no_grad():
                        features = self.model(img_tensor)
                return features.squeeze().numpy()
            print(f"Failed to download image from {url}")
        except Exception as e:
            print(f"Error processing image from {url}: {e}")
        return None

class DatasetManager:
    def __init__(self, dataset_path, feature_path):
        self.dataset_path = dataset_path
        self.feature_path = feature_path
        self.data = pd.read_csv(dataset_path)
        self.image_processor = ImageProcessor()
        self.data.fillna('', inplace=True)

    def process_images(self):
        if os.path.exists(self.feature_path):
            print("Features already exist. Loading...")
            return

        feature_dict = {}
        for index, row in self.data.iterrows():
            images = eval(row['Image'])
            features = [self.image_processor.fetch_and_process(url) for url in images if url]
            feature_dict[index] = [feature for feature in features if feature is not None]

        with open(self.feature_path, 'wb') as f:
            pickle.dump(feature_dict, f)
        print("Image processing complete.")

    def load_features(self):
        with open(self.feature_path, 'rb') as f:
            return pickle.load(f)

if __name__ == "__main__":
    dataset_path = '/content/drive/MyDrive/A2_Data.csv'
    feature_path = '/content/drive/MyDrive/image_features.pkl'

    manager = DatasetManager(dataset_path, feature_path)
    manager.process_images()
    features = manager.load_features()
    print("Feature extraction is done.")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Features already exist. Loading...
Feature extraction is done.


In [8]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from collections import Counter
import math
import pickle


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Simplified text preprocessing function
def preprocess_text(text):
    lower_text = text.lower()
    cleaned_text = ''.join(char for char in lower_text if char not in string.punctuation)
    tokens = word_tokenize(cleaned_text)
    filtered_tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return lemmatized_tokens

# Compute Term Frequency
def compute_tf(document):
    counts = Counter(document)
    return {word: count / len(document) for word, count in counts.items()}

# Compute Inverse Document Frequency
def compute_idf(word, corpus):
    return math.log10(len(corpus) / sum(1 for doc in corpus if word in doc))

# Compute TF-IDF for the entire corpus
def compute_tfidf(corpus):
    documents_tf = [compute_tf(doc) for doc in corpus]
    idf_values = {word: compute_idf(word, corpus) for doc in corpus for word in doc}
    tfidf_scores = [{word: tf * idf_values[word] for word, tf in doc_tf.items()} for doc_tf in documents_tf]
    return tfidf_scores

if __name__ == "__main__":
    # Specify file locations
    dataset_path = '/content/drive/MyDrive/A2_Data.csv'
    tfidf_path = '/content/drive/MyDrive/IR_ASSIGNMENT_2/tfidf.pkl'

    # Load dataset
    df = pd.read_csv(dataset_path)
    df['Review Text'].fillna('', inplace=True)

    # Preprocess reviews
    df['Processed Reviews'] = df['Review Text'].apply(preprocess_text)
    corpus = df['Processed Reviews'].tolist()

    # Compute and save TF-IDF scores
    tfidf_scores = compute_tfidf(corpus)
    with open(tfidf_path, 'wb') as file:
        pickle.dump(tfidf_scores, file)

    print("TF-IDF scores have been computed and saved.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


TF-IDF scores have been computed and saved.


In [23]:
#Q3


import pickle
import numpy as np
from scipy.spatial.distance import cosine
import pandas as pd
import nltk
import string
from collections import Counter
import math
import os
import requests
from PIL import Image
from torchvision import models, transforms
from io import BytesIO
import torch
from requests.exceptions import ConnectionError, HTTPError, Timeout

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

# Function to preprocess text
def textPreProcess(text):
    text = text.lower()
    noPuncText = ""
    for char in text:
        if char not in string.punctuation:
            noPuncText += char
    text = noPuncText
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stopwords]
    words = [lemmatizer.lemmatize(word) for word in words]
    return words

# Function to preprocess an image given its URL.
def imgPreProcess(image_url, maxTry=3):
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
    model.fc = torch.nn.Identity()
    model.eval()

    # Attempt to download and preprocess image up to maxTry times
    countTry = 0
    success = False
    while countTry < maxTry and not success:
        try:
            response = requests.get(image_url, timeout=10)  # Download image
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content)).convert('RGB')  # Open and convert image to RGB
                imgTrans = transform(img)  # Apply transformations
                imgTrans = imgTrans.unsqueeze(0)  # Add batch dimension
                with torch.no_grad():
                    features = model(imgTrans)  # Extract features
                features = features.view(features.size(0), -1)
                features = torch.nn.functional.normalize(features, p=2, dim=1)  # Normalize features
                success = True
                return features.cpu().numpy()
            else:
                print(f"Failed to download an image from {image_url}")
                return None
        except (ConnectionError, HTTPError, Timeout) as e:
            countTry += 1
            print(f"Attempt {countTry} failed for {image_url}: {e}")
    if not success:
        print(f"Failed to process image {image_url} after {maxTry} attempts.")
    return None

# Function to calculate Term Frequency (TF) from a document
def calcTf(document):
    textTf = Counter(document)
    for i in textTf:
        textTf[i] = textTf[i] / float(len(document))  # Calculate TF
    return textTf

# Define paths to your pickle files for image features and TF-IDF scores
imgFeaturesPath = '/content/drive/MyDrive/IR_ASSIGNMENT_2/image_features.pkl'
tfIdfPath = '/content/drive/MyDrive/IR_ASSIGNMENT_2/tfidf.pkl'

# Load precomputed image features and TF-IDF scores from pickle files
with open(imgFeaturesPath, 'rb') as f:
    image_features = pickle.load(f)
with open(tfIdfPath, 'rb') as f:
    tfScores = pickle.load(f)

# Function to find the top N similar images based on cosine similarity
def findSimImg(inputFeatures, imgFeatureDict, top=3):
    inputFeatures = np.squeeze(inputFeatures)
    similar = []
    for index, featureList in imgFeatureDict.items():
        for features in featureList:
            features = np.squeeze(features)
            similarity = 1 - cosine(inputFeatures, features)
            similar.append((index, similarity))

    similar.sort(key=lambda x: x[1], reverse=True)
    return similar[:top]

# Function to find the top N similar reviews based on cosine similarity
def findSimReview(inputTfIdf, tfidfList, top=3):
    inputTfVector = np.array([inputTfIdf.get(term, 0) for term in dicVocab])

    if not np.any(inputTfVector):
        print("Input review TF-IDF vector is all zeros. No similar reviews found.")
        return []

    similar = []
    for index, tfIdfDict in enumerate(tfidfList):
        tfidf_vector = np.array([tfIdfDict.get(term, 0) for term in dicVocab])
        if not np.any(tfidf_vector):
            continue

        similarity = 1 - cosine(inputTfVector, tfidf_vector)
        similar.append((index, similarity))

    similar.sort(key=lambda x: x[1], reverse=True)
    return similar[:top]

# Compute the dictionary of vocabulary from all TF-IDF scores for alignment
dicVocab = set()
for tfIdfDict in tfScores:
    dicVocab.update(tfIdfDict.keys())
dicVocab = sorted(dicVocab)

# Main program: Input preprocessing, find most similar images and reviews, and output results
inputImgUrl = input("Enter the image URL: ")
inputReview = input("Enter the review text: ")

imgProc = imgPreProcess(inputImgUrl)
reviewProc = textPreProcess(inputReview)
inputreviewTf = calcTf(reviewProc)

imgsSim = findSimImg(imgProc, image_features, top=3)
revsSim = findSimReview(inputreviewTf, tfScores, top=3)

print("Top 3 similar images:", imgsSim)
print("Top 3 similar reviews:", revsSim)

# Save results to a pickle file
results_path = '/content/drive/MyDrive/IR_ASSIGNMENT_2/sim_results.pkl'
retrieval_results = {
    'imgsSim': imgsSim,
    'revsSim': revsSim
}

with open(results_path, 'wb') as f:
    pickle.dump(retrieval_results, f)

print(f"Similar Results have been saved.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Enter the image URL: https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg
Enter the review text: I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.
Top 3 similar images: [(655, 0.796043336391449), (237, 0.7892007231712341), (939, 0.7771452069282532)]
Top 3 similar reviews: [(758, 0.9660084816861149), (622, 0.30610582916494966), (439, 0.17399008960531437)]
Similar Results have been saved.


In [28]:
#q4
import pandas as pd
import numpy as np
import pickle

# Specify the path to the stored similarity results
similarity_results_path = '/content/drive/MyDrive/IR_ASSIGNMENT_2/similarity_outcomes.pkl'

# Utility to fetch results from the designated pickle file
def retrieve_results(file_path):
    with open(simResultPath, 'rb') as data_file:
        results = pickle.load(data_file)
    return results['imgsSim'], results['revsSim']

# Generate composite scores from similarities and sort them
def compile_rankings(image_matches, text_matches):
    composite_list = []

    for img_match, txt_match in zip(image_matches, text_matches):
        image_idx, img_similarity = img_match
        text_idx, txt_similarity = txt_match

        # Average of similarities as composite score
        average_score = (img_similarity + txt_similarity) / 2
        composite_list.append((image_idx, text_idx, average_score))

    # Sorting the composite scores in descending order
    return sorted(composite_list, key=lambda item: item[2], reverse=True)[:3]

# Presentation of the ranked results
def present_ranked_outcomes(sorted_results):
    print("Summary of Composite Rankings:")
    for ranking, (img_id, rev_id, comp_score) in enumerate(sorted_results, start=1):
        print(f"Position: {ranking}, Image Ref: {img_id}, Review Ref: {rev_id}, Combined Score: {comp_score:.4f}")

# Main execution flow
if __name__ == "__main__":
    image_sims, review_sims = retrieve_results(similarity_results_path)
    sorted_composites = compile_rankings(image_sims, review_sims)
    present_ranked_outcomes(sorted_composites)

    # Storing the compiled and ranked results for future reference
    ranking_storage_path = '/content/drive/MyDrive/IR_ASSIGNMENT_2/compiled_rankings.pkl'
    with open(ranking_storage_path, 'wb') as storage:
        pickle.dump(sorted_composites, storage)
    print("The compiled rankings are now stored.")



Summary of Composite Rankings:
Position: 1, Image Ref: 655, Review Ref: 758, Combined Score: 0.8810
Position: 2, Image Ref: 237, Review Ref: 622, Combined Score: 0.5477
Position: 3, Image Ref: 939, Review Ref: 439, Combined Score: 0.4756
The compiled rankings are now stored.


In [6]:
#Q5

import pandas as pd
import numpy as np
import pickle

# Paths to the dataset and result files stored on the drive
dataLoc = '/content/drive/MyDrive/A2_Data.csv'
simResultLoc = '/content/drive/MyDrive/IR_ASSIGNMENT_2/sim_results.pkl'
rankResultLoc = '/content/drive/MyDrive/IR_ASSIGNMENT_2/rank_results.pkl'

# Load the dataset and the similarity and ranking results
df = pd.read_csv(dataLoc)  # Load the dataset containing images and review texts
with open(simResultLoc, 'rb') as file:  # Load similarity results
    simRes = pickle.load(file)
with open(rankResultLoc, 'rb') as file:  # Load ranking results
    rankRes = pickle.load(file)

# Function to calculate and print composite results using both image and text retrieval methods
def calCompositeResult(df, simRes, rankRes):
    # Print Image Retrieval Results
    print("\n")
    print("-----------------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------------")
    print("USING IMAGE RETRIEVAL")
    for i, (imgIndex, revIndex, compScore) in enumerate(rankRes, start=1):
        # Retrieve image URL and review text based on index from rank results
        urlImg = df.iloc[imgIndex]['Image']
        textReview = df.iloc[revIndex]['Review Text']

        # Retrieve cosine similarity scores for images and text from similarity results, ensuring indices are within bounds
        imgSim = simRes['imgsSim'][i-1][1] if i-1 < len(simRes['imgsSim']) else 0
        revSim = simRes['revsSim'][i-1][1] if i-1 < len(simRes['revsSim']) else 0

        # Print detailed results including URLs, review text, and similarity scores
        print(f"{i}) Image URL: {urlImg}")
        print(f"Review: {textReview}")
        print(f"Cosine Similarity of Images - {imgSim:.4f}")
        print(f"Cosine Similarity of Text - {revSim:.4f}")
        print(f"Composite Similarity Score: {compScore:.4f}\n")


    # Similar output structure for text retrieval results, but there seems to be a mistake as it repeats the image retrieval logic.
    print("-----------------------------------------------------------------------------------")
    print("USING TEXT RETRIEVAL")
    for i, (imgIndex, revIndex, compScore) in enumerate(rankRes, start=1):
        # Here, it incorrectly uses `revIndex` for both image and text, which likely is a mistake.
        urlImg = df.iloc[revIndex]['Image']
        textReview = df.iloc[revIndex]['Review Text']

        # Retrieve cosine similarity scores for images and text, with safeguards for index out-of-bounds
        imgSim = simRes['imgsSim'][i-1][1] if i-1 < len(simRes['imgsSim']) else 0
        revSim = simRes['revsSim'][i-1][1] if i-1 < len(simRes['revsSim']) else 0

        # Print results with URLs, review text, and similarity scores
        print(f"{i}) Image URL: {urlImg}")
        print(f"Review: {textReview}")
        print(f"Cosine Similarity of Images - {imgSim:.4f}")
        print(f"Cosine Similarity of Text - {revSim:.4f}")
        print(f"Composite Similarity Score: {compScore:.4f}\n")

    # Calculate and print composite similarity scores for both images and text, and their average as the final score
    imgSimilarities = [sim[1] for sim in simRes['imgsSim']]  # List of image similarity scores
    revSimilarities = [sim[1] for sim in simRes['revsSim']]  # List of text similarity scores
    ImgComscore = np.mean(imgSimilarities)  # Average image similarity score
    TextComscore = np.mean(revSimilarities)  # Average text similarity score
    finalScore = (ImgComscore + TextComscore) / 2  # Average of image and text scores
    print("-----------------------------------------------------------------------------------")

    # Print final composite similarity scores
    print("Composite similarity scores of images:", f"{ImgComscore:.4f}")
    print("Composite similarity scores of text:", f"{TextComscore:.4f}")
    print("Final composite similarity score:", f"{finalScore:.4f}\n")

    print("-----------------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------------")


calCompositeResult(df, simRes, rankRes)



-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
USING IMAGE RETRIEVAL
1) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/719-SDMiOoL._SY88.jpg']
Review: I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.
Cosine Similarity of Images - 0.7960
Cosine Similarity of Text - 0.9660
Composite Similarity Score: 0.8810

2) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/71vT2-nW7-L._SY88.jpg']
Review: I went from fender chrome non-locking to fender gold locking. It made my guitar look beautiful and play beautiful. I think locking tuners are the way to go. If you are new to locking tuners look on YouTube for instructions.
Cosine Similarity of Images - 0.7892
Cosine Similarity of Text - 0.3061
Composite Similarity Score: 0.547